In [ ]:
import torch
import numpy as np

import torch.nn.functional as F

from torch_geometric.loader import DataLoader
from utils.utilities import *
from utils.nop import *

import matplotlib.pyplot as plt

In [ ]:
# Data and model location
data_path = 'data/datasetTPN.mat'
model_path = 'models/model_optimized.pt'

device = torch.device('cuda')

In [ ]:
# Load model
model_data = torch.load(model_path, map_location=device)
pars = model_data['pars']

model = KernelNN(pars['model']['width'], pars['model']['kernel_width'], pars['model']['depth'], 1, in_width=2, out_width=2).to(device)
model.load_state_dict(model_data['model'])
model.eval()

In [ ]:
# Case for evaluation
test_case = 90
n_mesh = 2000
n_neighbors = 0
radius = 3

In [ ]:
# Load data and run model
d = data_loader(data_path)
test_data = d.sample_mesh(n_mesh, test_case, n_connections=n_neighbors, radius=radius, seed=test_case).to(device)
x = test_data.coords
y_0 = test_data.x.cpu().detach().numpy()
y_gt = test_data.y.cpu().detach().numpy()

with torch.no_grad():
    y_p = model(test_data).cpu().detach().numpy()

In [ ]:
# Plot results
n_contours = 16

plt.figure(figsize=(12,4))

contour_max = max(np.abs(np.hstack((y_0[:,0].flatten(),y_gt[:,0].flatten(),y_p[:,0].flatten()))))
contours = np.linspace(-contour_max,contour_max,n_contours)

plt.subplot(2,4,1)
plt.tricontourf(x[:,0],x[:,1],y_0[:,0],contours)
plt.title('Initial condition')
plt.ylabel('Surface height [m]')
plt.subplot(2,4,2)
plt.tricontourf(x[:,0],x[:,1],y_gt[:,0],contours)
plt.title('Ground truth')
plt.subplot(2,4,3)
plt.tricontourf(x[:,0],x[:,1],y_p[:,0],contours)
plt.title('Prediction')
plt.subplot(2,4,4)
plt.tricontourf(x[:,0],x[:,1],y_p[:,0]-y_gt[:,0],contours)
plt.title('Error')
plt.colorbar()

contour_max = max(np.abs(np.hstack((y_0[:,1].flatten(),y_gt[:,1].flatten(),y_p[:,1].flatten()))))
contours = np.linspace(-contour_max,contour_max,n_contours)

plt.subplot(2,4,5)
plt.tricontourf(x[:,0],x[:,1],y_0[:,1],contours)
plt.ylabel('Surface velocity [m/s]')
plt.subplot(2,4,6)
plt.tricontourf(x[:,0],x[:,1],y_gt[:,1],contours)
plt.subplot(2,4,7)
plt.tricontourf(x[:,0],x[:,1],y_p[:,1],contours)
plt.subplot(2,4,8)
plt.tricontourf(x[:,0],x[:,1],y_p[:,1]-y_gt[:,1],contours)
plt.colorbar()

In [ ]:
# Print metrics
def ioa(p,o):
    p = p.flatten()
    o = o.flatten()

    return 1 - (np.sum(np.square(o-p))/np.sum(np.square(np.abs(p-np.mean(o))+np.abs(o-np.mean(o)))))

print(f'MSE: {np.mean(np.square(y_p-y_gt))}')
print(f'MAE: {np.mean(np.abs(y_p-y_gt))}')
print(f'IOA: {ioa(y_gt,y_p)}')

In [ ]:
# Plot loss during training
plt.semilogy(model_data['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')